In [1]:
import boto3, os

In [2]:
os.environ['LEARNER_S3_BUCKETNAME_TEXT'] = 'bedrockproject'
os.environ['LAMBDALAYERVERSIONARN'] = "arn:aws:lambda:us-west-2:332848115362:layer:dlai-bedrock-jinja-layer:1"

In [3]:
from helpers.Lambda_Helper import Lambda_Helper
from helpers.S3_Helper import S3_Helper
from helpers.Display_Helper import Display_Helper

In [4]:
# deploy_function
# add_lambda_trigger
lambda_helper = Lambda_Helper()

In [5]:
# upload_file
# download_object 
# list_objects
s3_helper = S3_Helper


In [6]:
# text_file
# json_file
display_helper = Display_Helper()

In [7]:
bucket_name_text = os.environ['LEARNER_S3_BUCKETNAME_TEXT']
print(bucket_name_text)

bedrockproject


In [8]:
%%writefile prompt_template.txt
I need to summarize a conversation. The transcript of the conversation is between the <data> XML like tags.

<data>
{{transcript}}
</data>

The summary must contain a one word sentiment analysis, and a list of issues, problems or causes of friction
during the conversation. The output must be provided in JSON format shown in the following example. 

Example output:
{
    "version": 0.1,
    "sentiment": <sentiment>,
    "issues": [
        {
            "topic": <topic>,
            "summary": <issue_summary>,
        }
    ]
}

An `issue_summary` must only be one of:
{%- for topic in topics %}
 - `{{topic}}`
{% endfor %}

Write the JSON output and nothing more.

Here is the JSON output:

Overwriting prompt_template.txt


In [9]:
display_helper.text_file('prompt_template.txt')

prompt_template.txt:


LAMBDA FUNCTION


#############################################################
#
# This Lambda function is written to a file by the notebook 
# It does not run in the notebook!
#
#############################################################




In [10]:



#############################################################
#
# This Lambda function is written to a file by the notebook 
# It does not run in the notebook!
#
#############################################################



In [11]:
%%writefile lambda_function.py
import boto3
import json
from jinja2 import Template

s3_client = boto3.client('s3')
bedrock_runtime = boto3.client('bedrock-runtime','us-west-2')

def lambda_handler(event, context):
    bucket = event['Records'][0]['s3']['bucket']['name']
    key = event['Records'][0]['s3']['object']['key']

    if '-transcription.json' not in key:
        print("This demo only works with *-transcript.json.")
        return
    try:

        file_content = ""

        response = s3_client.get_object(Bucket=bucket,Key=key)

        file_content = response['Body'].read().decode('utf-8')
        transcript = extract_transcript_from_textract(file_content)
        print(f'Successfully read file {key} from Bucket {bucket}')
        print(f'Transcript is {transcript}')

        summary = bedrock_summarization(transcript)

        s3_client.put_object(
            Bucket = bucket,
            Key= 'result.txt',
            Body = summary,
            ContentType = 'text/plain'

        )
        

    




    except Exception as e:

        print(f"Error occurred: {e}")
        return {
            'statusCode': 500,
            'body': json.dumps(f"Error occurred: {e}")
        }
    
    return {
        'statusCode': 200,
        'body': json.dumps(f"Successfully summarized {key} from bucket {bucket}. Summary: {summary}")
    }







def bedrock_summarization(transcript):
    with open('prompt_template.txt','r') as file:
        template_string = file.read()

    data = {
        'transcript': transcript,
        'topics':['charges', 'location', 'availability']
    }

    template = Template(template_string)
    prompt = template.render(data)

    kwargs = {
        "modelId": "amazon.titan-text-express-v1",
        "contentType": "application/json",
        "accept": "*/*",
        "body": json.dumps(
            {
                "inputText": prompt,
                "textGenerationConfig": {
                    "maxTokenCount": 2048,
                    "stopSequences": [],
                    "temperature": 0, #We want the model to be more consistent
                    "topP": 0.9
                }
            }
        )
    }
    
    response = bedrock_runtime.invoke_model(**kwargs)

    summary = json.loads(response.get('body').read()).get('results')[0].get('outputText')    
    return summary

def extract_transcript_from_textract(file_content):

    transcript_json = json.loads(file_content)

    output_text = ""
    current_speaker = None

    items = transcript_json['results']['items']

    # Iterate through the content word by word:
    for item in items:
        speaker_label = item.get('speaker_label', None)
        content = item['alternatives'][0]['content']
        
        # Start the line with the speaker label:
        if speaker_label is not None and speaker_label != current_speaker:
            current_speaker = speaker_label
            output_text += f"\n{current_speaker}: "
        
        # Add the speech content:
        if item['type'] == 'punctuation':
            output_text = output_text.rstrip()  # Remove the last space
        
        output_text += f"{content} "
        
    return output_text
        

Overwriting lambda_function.py


In [12]:
lambda_helper.deploy_function(['lambda_function.py','prompt_template.txt'],function_name='LambdaFunctionSummarize')

Zipping function...
Looking for existing function...


Function LambdaFunctionSummarize exists. Updating code...
Function LambdaFunctionSummarize code updated: 2024-03-20T08:08:52.000+0000
Done.


In [13]:
lambda_helper.filter_rules_suffix = "json"
lambda_helper.add_lambda_trigger(bucket_name_text)

Using function name of deployed function: LambdaFunctionSummarize
Removed existing permission: s3-trigger-permission
Permission added with Statement: {
    "Sid": "s3-trigger-permission",
    "Effect": "Allow",
    "Principal": {
        "Service": "s3.amazonaws.com"
    },
    "Action": "lambda:InvokeFunction",
    "Resource": "arn:aws:lambda:us-west-2:332848115362:function:LambdaFunctionSummarize",
    "Condition": {
        "ArnLike": {
            "AWS:SourceArn": "arn:aws:s3:::bedrockproject"
        }
    }
}
Trigger added for bedrockproject -> LambdaFunctionSummarize


In [16]:
file_namee=''
s3_helper.upload_file(bucket_name_text, 'demo_transcript.json')


TypeError: S3_Helper.upload_file() missing 1 required positional argument: 'file_name'

In [ ]:
# s3_helper.list_objects(bucket_name_text)
# s3_helper.download_object(bucket_name_text, 'results.txt')
# display_helper.text_file('results.txt')

TypeError: S3_Helper.list_objects() missing 1 required positional argument: 'bucket_name'